In [ ]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
 
print(os.getcwd())

In [ ]:
driver = webdriver.Edge()

### Scrape Links for BNM Press Release (English & Malay)

##### Get English Content URL

In [ ]:
#list of pages where links of available press release

date=[]
title=[]
link=[]

for year in list(range(2015,2025)):
    url='https://www.bnm.gov.my/press-release-'+str(year)
    print(url)
    driver.get(url)
    soup=BeautifulSoup(driver.page_source)
    
    press_release_list=soup.find('table',attrs={'class':'Press-table table'}).find_all('tr')[1:]

    for press_release in press_release_list:
        date.append(press_release.find_all('p')[0].text)
        title.append(press_release.find_all('p')[1].text)
        link.append(press_release.find_all('p')[1])
        
    time.sleep(random.randint(5,10))      

df=pd.DataFrame({'date':date , 'title':title , 'link':link})
df['link']=df['link'].astype(str).str.extract('("https.+")')
df['link']=df['link'].apply(lambda x: x.replace('"',''))

df    


In [ ]:
# df.to_csv('link1en.csv',index=False)
# df.to_csv('link2en.csv',index=False)
# df.to_csv('link3en.csv',index=False) # final retries 2015-2025

##### Get PDF (English) Links within Content

In [ ]:
link=[]
pdf_links=[]

df=pd.read_csv('link1en.csv')

for url in df['link'].to_list():
    print(url)
    link.append(url)
    driver = webdriver.Edge()
    driver.get(url)
    time.sleep(random.randint(5,10))
    soup=BeautifulSoup(driver.page_source)
    
    # Find all 'a' tags (which define hyperlinks)   
    a_tags = soup.find_all('a')
    
    file=[]
    for tag in a_tags:
        try:
            check_pdf=re.search('documents.+\.pdf', tag['href'])[0]
            check_pdf=check_pdf.replace('%2F','/') 
            #check_pdf in tag['href']
            file.append(check_pdf)
        except:
            pass
    
    file=list(set(file))
    
    # Filter the 'a' tags to get only those that link to a .pdf file
    pdf_links.append(file)
    
    df=pd.DataFrame({'link':link, 'pdf_links':pdf_links})
    df.to_csv('link-pdf-en.csv',index=False) # ref file 2
    
    driver.close()

#### Get Malay Content URL

In [ ]:
date=[]
title=[]
link=[]


for year in list(range(2000,2025)):
    url='https://www.bnm.gov.my/press-release-'+str(year)
    #print(url)
    driver = webdriver.Edge()
    driver.get(url)
    
    time.sleep(random.randint(5,10))
    
    soup=BeautifulSoup(driver.page_source)
    
    link_bm = soup.find('a',attrs={'lang':'ms-MY'})
    link_bm = "https://www.bnm.gov.my"+(re.search('/.+languageId=ms_MY',str(link_bm)).group(0).replace('&amp;','&'))
    
    driver.get(link_bm)
    
    time.sleep(random.randint(5,10))
    
    soup=BeautifulSoup(driver.page_source)
    
    press_release_list=soup.find('table',attrs={'class':'Press-table table'}).find_all('tr')[1:]

    for press_release in press_release_list:
        date.append(press_release.find_all('p')[0].text)
        title.append(press_release.find_all('p')[1].text)
        link.append(press_release.find_all('p')[1])
        
    driver.close()

    
df=pd.DataFrame({'date':date , 'title':title , 'link':link})
df['link']=df['link'].astype(str).str.extract('("https.+")')
df['link']=df['link'].apply(lambda x: x.replace('"',''))
df['link']=df['link'].str.replace(r'(->.+)','-',regex=True)
df.to_csv('link1ms.csv',index=False)
df 

#### Get PDF (Malay) Links within Content

In [ ]:
link=[]
pdf_links=[]

df=pd.read_csv('link1ms.csv')

#link=pd.read_csv('link-pdf-ms.csv')['link'].to_list()
#pdf_links=pd.read_csv('link-pdf-ms.csv')['pdf_links'].to_list()

#for url in df['link'].to_list():

for url in tqdm([x for x in df['link'].to_list() if x not in link]):
    try:
        #print(url)
        link.append(url)
        driver = webdriver.Edge()
        driver.get(url)
        time.sleep(random.randint(5,10))
        soup=BeautifulSoup(driver.page_source)
        
        
        link_bm = soup.find('a',attrs={'lang':'ms-MY'})
        link_bm = "https://www.bnm.gov.my"+(re.search('/.+languageId=ms_MY',str(link_bm)).group(0).replace('&amp;','&'))
        print(link_bm)
        link_bm = link_bm.replace('%2F','/')
        
        driver.get(link_bm)
        time.sleep(random.randint(5,6))
        
        soup=BeautifulSoup(driver.page_source)
        
        # Find all 'a' tags (which define hyperlinks)   
        a_tags = soup.find_all('a')
        
        file=[]
        for tag in a_tags:
            try:
                check_pdf=re.search('documents.+\.pdf', tag['href'])[0]
                #print(check_pdf)
                check_pdf=check_pdf.replace('%2F','/') 
                #check_pdf in tag['href']
                file.append(check_pdf)
                print(file)
            except:
                continue
        
        file=list(set(file))
        
        # Filter the 'a' tags to get only those that link to a .pdf file
        pdf_links.append(file)
        
        df=pd.DataFrame({'link':link, 'pdf_links':pdf_links})
        df.to_csv('link-pdf-ms.csv',index=False) # ref file 4
        
        driver.close()
        
    except:
        
        print('failed at ',url)
        
        file=[]
        pdf_links.append(file)
        
        df=pd.DataFrame({'link':link, 'pdf_links':pdf_links})
        df.to_csv('link-pdf-ms.csv',index=False) # ref file 4
        
        driver.close()
        time.sleep(random.randint(5, 10))

    

#### Pdf links ref. table (Malay + English)

In [ ]:
pdf_links=pd.concat([pd.merge(pd.read_csv('link1ms.csv'),pd.read_csv('link-pdf-ms.csv'),how='left',on='link'),pd.merge(pd.read_csv('link1en.csv'),pd.read_csv('link-pdf-en.csv'),how='left',on='link')]).reset_index(drop=True)
pdf_links=pdf_links[~(pdf_links['link'].str.contains('%'))]
pdf_links=pdf_links.drop_duplicates('link').reset_index(drop=True)
pdf_links['link']=np.where(pdf_links['link'].str.endswith('.'),pdf_links['link'].str[:-1],pdf_links['link'])
pdf_links


In [ ]:
date=[]
title=[]
link=[]
links=[]

for x in list(range(len(pdf_links))):
    
    date.append(pdf_links['date'][x])
    title.append(pdf_links['title'][x])
    link.append(pdf_links['link'][x])
    
    try:
        
        pdf=""

        for row_link in re.search(r"'documents.+\.pdf'",pdf_links['pdf_links'][x])[0].replace("'","").split(","):
            pdf=pdf+'https://www.bnm.gov.my/'+row_link.strip()+", "

        pdf=pdf[:-2]
        
        links.append(pdf)
        
    except:
        
        pdf=""
        links.append(pdf)
    
df=pd.DataFrame({'date':date, 'title':title, 'link':link, 'pdf_links':links})

pdf_links=pd.merge(pdf_links.drop(columns=['pdf_links']), df, on=['date','title','link'], how='left')
pdf_links=pdf_links.drop_duplicates().reset_index(drop=True)
pdf_links



### Scrape Content Based on Links (English & Malay)

In [ ]:
# concatenate all links (english) csv files

files=os.listdir()
files=list(filter(lambda k: 'en.csv' in k, files))

df=pd.DataFrame()

for file in files:
    df=pd.concat([df,(pd.read_csv(file))])

content=[]
link=[]

for url in tqdm([x for x in df['link'].to_list() if x not in link]):
    try:
        print(url)
        driver.get(url)
        
        time.sleep(random.randint(5,10))
        
        data=driver.page_source
        soup=BeautifulSoup(data)
        
        content.append(soup.find('div',attrs={'class':'journal-content-article'}).text)
        link.append(url)
        
        dfContent=pd.DataFrame({'link':link , 'content':content})
        dfContent.to_csv('content1.csv', index=False)
    except:
        print('failed at ',url)
        time.sleep(random.randint(10, 20))
        continue


In [ ]:
press_release_english=pd.merge(df,dfContent,left_on='link',right_on='link')
press_release_english.to_csv('press_release_english.csv',index=False)

In [ ]:
df=pd.read_csv('link1ms.csv')

#link=pd.read_csv('content2.csv')['link'].to_list()
#link_opt=pd.read_csv('content2.csv')['link_opt'].to_list()
#content=pd.read_csv('content2.csv')['content'].to_list()

#link=link
#link_opt=link_opt
#content=content

content=[]
link=[]
link_opt=[]


for url in tqdm([x for x in df['link'].to_list() if x not in link]):
    try:
        print(url)
        driver = webdriver.Edge()
        driver.get(url)
        
        time.sleep(random.randint(5,6))
        
        data=driver.page_source
        soup=BeautifulSoup(data)
        
        
        link_bm = soup.find('a',attrs={'lang':'ms-MY'})
        link_bm = "https://www.bnm.gov.my"+(re.search('/.+languageId=ms_MY',str(link_bm)).group(0).replace('&amp;','&'))
        
        driver.get(link_bm)
        
        time.sleep(random.randint(5,6))
        
        soup=BeautifulSoup(driver.page_source)
        
        content.append(soup.find('div',attrs={'class':'journal-content-article'}).text)
        link.append(url)
        link_opt.append(link_bm)
        
        driver.close()
        
        df=pd.DataFrame({'link':link , 'link_opt':link_opt , 'content':content})
        df.to_csv('content2.csv',index=False)

    except:
        print('failed at ',url)
        time.sleep(random.randint(10,20))
        continue




In [ ]:
(pd.merge(pd.read_csv('link1en.csv'),pd.read_csv('content1.csv'),how='left',on='link')).to_csv('press_release_english.csv',index=False)
(pd.merge(pd.read_csv('link1ms.csv'),pd.read_csv('content2.csv'),how='left',on='link')).to_csv('press_release_malay.csv',index=False)



#### Combine English & Malay Content + Add in Category Column as "Press Release" 

In [ ]:
bnm_data=pd.concat([pd.read_csv('press_release_english.csv'),pd.read_csv('press_release_malay.csv')])
bnm_data['category']='Press Release'
bnm_data=bnm_data.reset_index(drop=True)
bnm_data['link']=bnm_data['link'].str.strip()
bnm_data=bnm_data[~(bnm_data['link'].str.contains('%'))]
bnm_data=bnm_data.drop_duplicates('link')
bnm_data

In [ ]:
bnm_data['content']=bnm_data['content'].str.replace('\r\n\r\n\r\n\r\n','')
bnm_data['content']=bnm_data['content'].str.replace('\r\n\r\n\r\n','\r\n')
bnm_data['content']=bnm_data['content'].str.replace('\r\n\r\n\r\n\r\n\r\n\r\n\r\n','')
bnm_data['content']=bnm_data['content'].str.replace('Shares\r\n','')
bnm_data['content']=bnm_data['content'].str.replace('  Share:','Share:')
bnm_data['content']=bnm_data['content'].str.replace('      ','')
bnm_data


In [ ]:
bnm_data=pd.merge(pdf_links,bnm_data,how='left',left_on=['date','title','link'],right_on=['date','title','link']).drop_duplicates()
bnm_data

### Download PDF files

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import pyautogui
import pyperclip


link = []
pdf_links = []

# Filter the DataFrame once and reset the index

'''
filtered_data = bnm_data[bnm_data['pdf_links'] != ''].reset_index(drop=True)
downloaded=pd.DataFrame({'link':link,'pdf_links':pdf_links})[['link']].drop_duplicates()
downloaded['tag']='yes'
filtered_data=pd.merge(filtered_data,downloaded,how='left',on='link')
filtered_data=filtered_data[filtered_data['tag'].isnull()]
filtered_data=filtered_data.drop(columns=['tag'])
filtered_data=filtered_data.reset_index(drop=True)
'''

filtered_data = bnm_data[bnm_data['pdf_links'] != ''].reset_index(drop=True)
filtered_data = filtered_data[['link','pdf_links']]
filtered_data['pdf_links']=(filtered_data['pdf_links'].str.split(', ',expand=False))
filtered_data = filtered_data.explode('pdf_links')
filtered_data['filename']=filtered_data['pdf_links'].str.split('/').str[-1]
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/press_release/pdf/')
filtered_data=pd.merge(filtered_data,pd.DataFrame({'filename':os.listdir(),'downloaded':'yes'}),how='left',on='filename')
filtered_data=filtered_data[filtered_data['downloaded'].isnull()].drop(columns=['downloaded'])
filtered_data=filtered_data.reset_index(drop=True)

# Setup Edge options
edge_options = Options()

# Initialize the driver once
driver = webdriver.Edge(options=edge_options)

for row in tqdm(range(len(filtered_data))):
    current_link = filtered_data['link'][row]
    pdf_list = filtered_data['pdf_links'][row].split(', ')
    
    for pdf in pdf_list:
        
        try:
        
            link.append(current_link)
            pdf_links.append(pdf)
            driver.get(pdf)
            
            time.sleep(30)  # Wait for the page to load
            pyautogui.hotkey('ctrl', 's')  # Simulate Ctrl + S (save)
            time.sleep(10)  # Wait for the save dialog to appear
            path_and_filename = 'c:\\Users\\izard\\Documents\\GitHub\\bimb-osint-development\\back-end\\bnm\\press_release\\pdf\\'+pdf.split('/')[-1]
            pyperclip.copy(path_and_filename)
            pyautogui.hotkey('ctrl', 'v')
            pyautogui.press('enter')  # Press Enter to save the file
            pyautogui.press('enter')  # Press Enter to save the file
            
            time.sleep(random.randint(10,20))
            
            print('success')
        
        except:
            print('unsuccess')
            pass

# Close the driver after processing all rows
driver.quit()


### PDF to Text

In [ ]:
pdf_files = bnm_data[bnm_data['pdf_links'] != ''].reset_index(drop=True)
pdf_files = pdf_files[['link','pdf_links']]
pdf_files['pdf_links']=(pdf_files['pdf_links'].str.split(', ',expand=False))
pdf_files = pdf_files.explode('pdf_links')
pdf_files['filename']=pdf_files['pdf_links'].str.split('/').str[-1]
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/press_release/pdf/')
pdf_files=pd.merge(pdf_files,pd.DataFrame({'filename':os.listdir(),'downloaded':'yes'}),how='left',on='filename')
pdf_files=pdf_files[pdf_files['downloaded'].notnull()].drop(columns=['downloaded'])
pdf_files=pdf_files.reset_index(drop=True)
pdf_files

In [ ]:
# start tika server , check at http://localhost:9998/

os.chdir('/Users/izard/') # change directory to where tika-server-standard-2.6.0.jar is located

! java -jar tika-server-standard-2.6.0.jar

In [ ]:
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/press_release/pdf/')
os.getcwd()

In [ ]:
import tika
from tika import parser
from tqdm import tqdm

downloaded_files=os.listdir() # get list of downloaded files

tika.initVM()
print('start..')

filename=[]
Document_Text=[]

for file in tqdm(downloaded_files):

    try:
        print('parsing...',file)
        
        text=parser.from_file(file)['content']
        
        filename.append(file)
        Document_Text.append(text)
                
    except:
        print('unsuccess...',file)
        filename.append(file)
        Document_Text.append('')
        pass
    
pdf_to_text=pd.DataFrame({'filename':filename, 'Document_Text':Document_Text})

#### Update pdf_files with text extracted columns

In [ ]:
pdf_files=pd.merge(pdf_files,pdf_to_text,how='left',on='filename')
pdf_files

#### Filter Only to PDF to Text Valid Rows

In [ ]:
pdf_files['Document_Text']=np.where((pdf_files['Document_Text']=='')|(pdf_files['Document_Text'].isnull()),'N/A',pdf_files['Document_Text'])

#### Concat PDF to Text Extracted with Same Link

In [ ]:
pdf_files=pdf_files[['link','Document_Text']].groupby(['link']).agg({'Document_Text':'sum'}).reset_index()
pdf_files

#### Update Main Dataframe

In [ ]:
bnm_data=pd.merge(bnm_data,pdf_files,how='left',on='link')[['date', 'title', 'link', 'pdf_links', 'link_opt', 'content', 'Document_Text', 'category']]
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/press_release/')
bnm_data.to_csv('bnm_data_press_release.csv',index=False)
bnm_data

#### Convert to JSON

In [40]:
# standardize the timestamp
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/press_release/')
from datetime import datetime,timedelta

bnm_data=pd.read_csv('bnm_data_press_release.csv')
bnm_data['date']=np.where(bnm_data['date'].str.contains('Dis'),bnm_data['date'].str.replace('Dis','Dec'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Okt'),bnm_data['date'].str.replace('Okt','Oct'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Ogos'),bnm_data['date'].str.replace('Ogos','Aug'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Mei'),bnm_data['date'].str.replace('Mei','May'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Mac'),bnm_data['date'].str.replace('Mac','Mar'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Mac'),bnm_data['date'].str.replace('Mac','Mar'),bnm_data['date'])
bnm_data['date']=pd.to_datetime(bnm_data['date'])

In [41]:
bnm_data=bnm_data.rename(columns={'date':'Date_created','title':'Title','link':'URL','pdf_links':'pdf_URL','link_opt':'URL_opt','content':'Content','category':'Category'})
bnm_data

,Date_created,Title,URL,pdf_URL,URL_opt,Content,Document_Text,Category
0,2000-12-31,Penggabungan dan Pengukuhan Institusi Perbanka...,https://www.bnm.gov.my/-/penggabungan-dan-peng...,NaN,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nPenggabungan dan Pengukuha...,NaN,Press Release
1,2000-12-30,Makluman Terperinci bagi Rizab Antarabangsa pa...,https://www.bnm.gov.my/-/makluman-terperinci-b...,NaN,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nMakluman Terperinci bagi R...,NaN,Press Release
2,2000-12-26,Expansion and Enlargement of the ASEAN Swap Ar...,https://www.bnm.gov.my/-/expansion-and-enlarge...,NaN,NaN,NaN,NaN,NaN
3,2000-12-22,Rizab Antarabangsa BNM pada 15 Dis 2000,https://www.bnm.gov.my/-/rizab-antarabangsa-bn...,NaN,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nRizab Antarabangsa BNM pad...,NaN,Press Release
4,2000-12-08,Rizab Antarabangsa BNM pada 30 Nov 2000,https://www.bnm.gov.my/-/rizab-antarabangsa-bn...,NaN,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nRizab Antarabangsa BNM pad...,NaN,Press Release
...,...,...,...,...,...,...,...,...
4671,2024-01-26,Memorandum of Understanding between Bank Negar...,https://www.bnm.gov.my/-/mou-csm,NaN,NaN,\r\n\r\nReading:\r\nMemorandum of Understandin...,NaN,Press Release
4672,2024-01-24,Monetary Policy Statement,https://www.bnm.gov.my/-/monetary-policy-state...,https://www.bnm.gov.my/documents/20124/1323546...,NaN,\r\n\r\nReading:\r\nMonetary Policy Statement\...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Press Release
4673,2024-01-22,International Reserves of Bank Negara Malaysia...,https://www.bnm.gov.my/-/international-reserve...,https://www.bnm.gov.my/documents/20124/6118085...,NaN,\r\n\r\nReading:\r\nInternational Reserves of ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Press Release
4674,2024-01-22,Statement on Claims Relating to Quantum Metal ...,https://www.bnm.gov.my/-/pr-qmsb,https://www.bnm.gov.my/documents/20124/60360/F...,NaN,\r\n\r\nReading:\r\nStatement on Claims Relati...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Press Release


In [42]:
out=bnm_data.to_json(orient = "records",date_format='iso')

with open('bnm_data_press_release.json', 'w') as f:
       f.write(out)